In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
%pip install efficientnet-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2 
from tqdm import tqdm
from torchvision import models
from efficientnet_pytorch import EfficientNet
from sklearn import metrics

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [36]:
class Dataset(Dataset):
    def __init__(self, root_images, root_file, transform = None):
        self.root_images = root_images
        self.root_file = root_file
        self.transform = transform
        self.file = pd.read_csv(root_file)


    def __len__(self):
        return self.file.shape[0]
    
    def __getitem__(self,index):
        img_path = os.path.join(self.root_images, self.file['id'][index])
        image = np.array(Image.open(img_path).convert('RGB'))
    
        if self.transform is not None:
            augmentations = self.transform(image = image)
            image = augmentations['image'] 
        
        return image

In [37]:
learning_rate = 0.0001
batch_size = 32
epochs = 10
height = 224 
width = 224
IMG = '/content/drive/MyDrive/Colab Notebooks/AI images or Not/test'
FILE = '/content/sample_submission.csv'

In [38]:
def get_loader(image, file, batch_size, test_transform):
    
    test_ds = Dataset(image , file, test_transform)
    test_loader = DataLoader(test_ds, batch_size= batch_size, shuffle= False)



    return test_loader 

In [39]:
normalize = A.Normalize(
    mean = [0.485 , 0.456 , 0.406],
    std = [0.229 , 0.224, 0.255],
    max_pixel_value= 255.0
)


test_transform = A.Compose(
    [A.Resize(width=width , height= height),
    normalize,
    ToTensorV2()
    ]
)


In [40]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b4')
        self.fct = nn.Linear(1000,1)
    
    def forward(self,img):
        x = self.model(img)
        # print(x.shape)
        x = self.fct(x)
        return x

In [41]:
def load_checkpoint(checkpoint, model, optimizer):
    print('====> Loading...')
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

In [42]:
test = pd.read_csv(FILE)
test

,id,label
0,0.jpg,0
1,1.jpg,0
2,10.jpg,0
3,100.jpg,0
4,1000.jpg,0
...,...,...
43437,9995.jpg,0
43438,9996.jpg,0
43439,9997.jpg,0
43440,9998.jpg,0


In [43]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr= learning_rate)

checkpoint_file = '/content/drive/MyDrive/Colab Notebooks/AI images or Not/baseline_V0.pth.tar'
test_loader = get_loader(IMG, FILE, batch_size, test_transform)
checkpoint = torch.load(checkpoint_file, map_location=torch.device('cpu'))
load_checkpoint(checkpoint, model, optimizer)

model.eval()
k = 0
for x in tqdm(test_loader):
  x = x.to(device).to(torch.float32)
  p = torch.sigmoid(model(x)).cpu().detach().numpy()

  for i in range(len(p)):
    test['label'][k] = (p[i] > 0.75).astype('float')
    k += 1

Loaded pretrained weights for efficientnet-b4
====> Loading...


  0%|          | 0/1358 [00:00<?, ?it/s]<ipython-input-43-383dee41b09a>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['label'][k] = (p[i] > 0.75).astype('float')
100%|██████████| 1358/1358 [4:56:02<00:00, 13.08s/it]


In [44]:
test

,id,label
0,0.jpg,0
1,1.jpg,0
2,10.jpg,0
3,100.jpg,1
4,1000.jpg,0
...,...,...
43437,9995.jpg,1
43438,9996.jpg,0
43439,9997.jpg,0
43440,9998.jpg,0


In [45]:
test.to_csv('sub.csv', index=False)

In [46]:
def predict(image):
    image = np.array(image)
    transform = A.Compose(
        [A.Resize(width=width, height=height),
         normalize,
         ToTensorV2()
         ]
    )
    image = transform(image=image)["image"].unsqueeze(0).to(device).to(torch.float32)
    with torch.no_grad():
        model.eval()
        output = torch.sigmoid(model(image))
        label = (output > 0.75).item()
    return "AI Image" if label else "Not AI Image"

In [47]:
%pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.7/129.7 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 k

In [48]:
import gradio as gr

In [49]:


inputs = gr.inputs.Image()
outputs = gr.outputs.Textbox()
iface = gr.Interface(fn=predict, inputs=inputs, outputs=outputs, capture_session=True)
iface.launch()

/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `capture_session` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [55]:
import gradio as gr
import torch
import numpy as np
from PIL import Image

# define the predict function
def predict(image):
    # preprocess the image
    image = np.array(image)
    image = test_transform(image=image)['image']
    image = image.unsqueeze(0).to(device)

    # get the model prediction
    with torch.no_grad():
        output = model(image)
        pred = torch.sigmoid(output).cpu().numpy().squeeze()
    
    # return the prediction as a string
    return f"This image is {'AI generated' if pred > 0.75 else 'NOT AI generated'}"

# define the input interface with examples
inputs = gr.inputs.Image(shape=(224, 224))
outputs = gr.outputs.Textbox()
examples = [
    ['/content/drive/MyDrive/Colab Notebooks/AI images or Not/train/3.jpg'],
    ['/content/drive/MyDrive/Colab Notebooks/AI images or Not/train/10.jpg'],
    ['/content/drive/MyDrive/Colab Notebooks/AI images or Not/train/14.jpg'],
    ['/content/drive/MyDrive/Colab Notebooks/AI images or Not/train/4515.jpg']
    ['/content/drive/MyDrive/Colab Notebooks/AI images or Not/train/4518.jpg'],
]
iface = gr.Interface(fn=predict, inputs=inputs, outputs=outputs, examples=examples)

# launch the gradio app
iface.launch()

<>:28: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:28: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
<ipython-input-55-ad9875932060>:28: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  ['/content/drive/MyDrive/Colab Notebooks/AI images or Not/train/4

TypeError: ignored